In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/ecommerce-data/data.csv',encoding='cp874')
df

In [ ]:
df_time = df.dropna()
df_time = df_time[df_time['Quantity']>0]
df_time = df_time[df_time['UnitPrice']>0]
df_1 = df_time[~df_time['StockCode'].str.contains('POST')]
df_1

In [ ]:
df_cus = df_1.loc[:,['InvoiceNo','CustomerID']]
df_cus

In [ ]:
df_dp = df_cus.drop_duplicates()
df_dp

In [ ]:
frequency = df_dp.groupby(['CustomerID']).count()
frequency = frequency.reset_index()
df_fre = frequency.rename(columns = {'InvoiceNo':'Frequency'})
df_fre.head()

In [ ]:
df_fre.sort_values('Frequency',ascending= False).head()
df_fre.describe()

In [ ]:
df_time = df_1.loc[:,['CustomerID','InvoiceDate']]
df_time

In [ ]:
df_time = df_time.drop_duplicates()
df_time

In [ ]:
from datetime import datetime
from datetime import date
from datetime import time

In [ ]:
df_time.InvoiceDate = pd.to_datetime(df_time.InvoiceDate)

In [ ]:
df_time = df_time.sort_values('InvoiceDate',ascending=False)
df_time = df_time.drop_duplicates(subset = 'CustomerID')
df_time['Timenow'] = datetime(2012,1,19,23,10)
df_time

In [ ]:
df_time['Recency'] = df_time.Timenow - df_time.InvoiceDate
df_time = df_time.sort_values('CustomerID')
df_time

In [ ]:
num = []
for i in df_time.Recency :
    num.append(i.days)
print(num)

In [ ]:
df_time['Recency'] = num
df_time

In [ ]:
df_recency = df_time.loc[:,['CustomerID','Recency']]
df_recency

In [ ]:
df_mty = df_1.loc[:,['CustomerID','InvoiceNo','UnitPrice','Quantity']]
df_mty

In [ ]:
df_mty['Monetary'] = df_mty['UnitPrice'] * df_mty['Quantity']
df_mty

In [ ]:
monetary = df_mty.groupby('CustomerID')[['Monetary']].sum()
monetary = monetary.reset_index()
monetary

In [ ]:
df_final = df_fre.merge(monetary)
df_final['BasketSize'] = (df_final['Monetary']/df_final['Frequency']).round(2)
df_final

In [ ]:
df_final = df_final.merge(df_recency)
df_final

In [ ]:
df_final.describe()

In [ ]:
q3 = df_final.Frequency.quantile(.75)
q1 = df_final.Frequency.quantile(.25)
iqr = q3-q1
print('Q1 : {}'.format(q1))
print('Q3 : {}'.format(q3))
print('IQR : {}'.format(iqr))

upper = q3 + (2*iqr)
lower = q1 - (1.5*iqr)
display('Lower : {:.2f} Upper : {:.2f}'.format(lower,upper))

In [ ]:
filter = df_final.Frequency > upper
df_final[filter]

In [ ]:
df_last = df_final[df_final.Frequency <= upper]
df_last.CustomerID = df_last.CustomerID.astype('category')
df_last

In [ ]:
df_last.describe()

In [ ]:
df_last = df_last[df_last.Monetary <= 10000]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples

In [ ]:
X = ['Frequency', 'Monetary', 'BasketSize', 'Recency']

In [ ]:
df_input = df_last[X]
df_input

In [ ]:
sils = []
for i in range(2,7) :
    model = KMeans(n_clusters = i)
    y_kmeans = model.fit(df_input)
    sil_avg = silhouette_score(df_input,model.labels_).round(4)
    sils.append([sil_avg,i])
print(sils)

In [ ]:
model = KMeans(n_clusters = 5)
y_kmeans = model.fit_predict(df_input)

In [ ]:
df_last['Cluster'] = y_kmeans
df_last

In [ ]:
df_last.Cluster.value_counts()

In [ ]:
b = df_last.groupby('Cluster').mean()
b

In [ ]:
b['Count'] = df_last.Cluster.value_counts()
b

In [ ]:
b['Group Name'] = ['Bronze','Platinum','Silver','Gold','Diamond']
b

cluster 4 or Group name Diamond is the best customer cluster.